In [7]:
import copy

bays = [[1,2],[],[4],[5, 6]]

crane_position = 0
crane_container_held = 3
initial_cost = 0


def print_state(state_and_time):
    state = state_and_time[0]
    for i in range(len(state[0])):
        if i == state[1]:
            print(str(state[0][i])+" <-["+str(state[2])+"]-")
        else:
            print(str(state[0][i]))
    print("Total cost: "+str(state_and_time[1]))
    print()


initial_state = ([bays, crane_position, crane_container_held], 0)
print_state(initial_state)

[1, 2] <-[3]-
[]
[4]
[5, 6]
Total cost: 0



In [8]:
def perform_action(state,action):
  new_state = copy.deepcopy(state[0])
  new_state_cost = state[1]
  cost = 0
  
  if action == "RIGHT":
    if new_state[1] >= (len(new_state[0]) -1):
      return None
    new_state[1] = int(new_state[1])+1
    cost = 1
  if action == "LEFT":
    if new_state[1] <= 0:
      return None
    new_state[1] = int(new_state[1])-1
    cost = 1
  if action == "DROP":
    if new_state[2] == None:
      return None
    container = new_state[2]
    new_state[2] = None
    new_state[0][new_state[1]].append(container)
    cost = 2
  if action == "PICK":
    if new_state[2] != None:
      return None
    if len(new_state[0][new_state[1]]) == 0:
      return None
    container = new_state[0][new_state[1]].pop()
    new_state[2] = container
    cost = 3
  return new_state, new_state_cost+cost

def perform_action_sequence(state,actions):
  new_state = state
  for action in actions:
    new_state = perform_action(new_state, action)
  return new_state


print_state(perform_action(initial_state,"RIGHT"))
print_state(perform_action(initial_state,"DROP"))
print_state(perform_action_sequence(initial_state,["RIGHT","DROP", "PICK", "RIGHT", "RIGHT", "LEFT", "RIGHT"]))
print_state(perform_action_sequence(initial_state,["RIGHT","DROP","RIGHT","PICK"]))

[1, 2]
[] <-[3]-
[4]
[5, 6]
Total cost: 1

[1, 2, 3] <-[None]-
[]
[4]
[5, 6]
Total cost: 2

[1, 2]
[]
[4]
[5, 6] <-[3]-
Total cost: 10

[1, 2]
[3]
[] <-[4]-
[5, 6]
Total cost: 7



In [ ]:
import copy
import time
import random

all_actions = ["DROP", "PICK", "LEFT", "RIGHT"]

def action_validity(state, action):
    if perform_action(state, action):
        return True
    else:
        return False
    
def is_goal_state(state):
    for bay in state[0]:
        if len(bay) == 0:
            return False
        return True
    
    